In [1]:
import pandas as pd

In [9]:
cons = pd.read_csv("consultations_time.csv", encoding="utf-8-sig", parse_dates=["start_time","end_time","consultation_date"])
cft  = pd.read_csv("consultation_fund_types.csv", encoding="utf-8-sig")
ft   = pd.read_csv("fund_types.csv", encoding="utf-8-sig")

In [ ]:
# 시간대×요일 상담 건수 예측 준비 (X, y 만들기)

import numpy as np

def make_hourly_design(cons: pd.DataFrame):
    df = cons.copy()
    df["ts_hour"] = df["start_time"].dt.floor("H")
    # y: 시간별 건수
    y_hourly = df.groupby("ts_hour").size().rename("y").reset_index()

    # 연속 시간축
    ts_full = pd.DataFrame({
        "ts_hour": pd.date_range(y_hourly["ts_hour"].min().floor("H"),
                                 y_hourly["ts_hour"].max().ceil("H"),
                                 freq="H")
    })
    Xy = ts_full.merge(y_hourly, on="ts_hour", how="left").fillna({"y":0})

    # 시간/요일 파생
    Xy["weekday"] = Xy["ts_hour"].dt.weekday
    Xy["hour"]    = Xy["ts_hour"].dt.hour

    # 주기 인코딩(24h)
    Xy["hour_sin"] = np.sin(2*np.pi*Xy["hour"]/24)
    Xy["hour_cos"] = np.cos(2*np.pi*Xy["hour"]/24)

    # 라그/롤링
    for h in [1, 24, 168]:  # 1시간, 하루, 1주
        Xy[f"lag_{h}"] = Xy["y"].shift(h)
    Xy["roll_3h"]  = Xy["y"].rolling(3,  min_periods=1).mean()
    Xy["roll_24h"] = Xy["y"].rolling(24, min_periods=1).mean()

    # 요일 원-핫(0~6)
    Xy = pd.concat([Xy, pd.get_dummies(Xy["weekday"], prefix="wd", drop_first=False)], axis=1)

    # 학습 행(결측 제거)
    Xy = Xy.dropna().reset_index(drop=True)

    y = Xy["y"].astype(float).values
    drop_cols = ["ts_hour","y","weekday","hour"]
    X = Xy.drop(columns=drop_cols).astype(float)

    meta = Xy[["ts_hour","weekday","hour"]].copy()
    return meta, X, y

In [ ]:
# 시간대×요일 최다 자금유형 분류 준비 (X, y 만들기)

def make_topfund_design(cons: pd.DataFrame, cft: pd.DataFrame, ft: pd.DataFrame, topK=6):
    """슬롯별 최다 자금유형 라벨 만들고, 시간/요일/지난주라벨 피처 구성"""
    df = cons.copy()
    df["ts_hour"] = df["start_time"].dt.floor("H")

    # 슬롯별 fund 분포
    cf = (cft.merge(df[["consultation_id","ts_hour"]], on="consultation_id", how="inner")
             .merge(ft, on="fund_type_id", how="left"))

    slot_fund = (cf.groupby(["ts_hour","fund_type_name"])
                   .size().rename("cnt").reset_index())

    # 슬롯별 최다 라벨
    slot_top = (slot_fund.sort_values(["ts_hour","cnt"], ascending=[True,False])
                         .drop_duplicates("ts_hour")
                         .rename(columns={"fund_type_name":"label"})
                         [["ts_hour","label"]])

    # 베이스(연속 시간축 → 실제 라벨 있는 슬롯만 사용)
    base = pd.DataFrame({"ts_hour": pd.date_range(df["start_time"].min().floor("H"),
                                                  df["start_time"].max().ceil("H"),
                                                  freq="H")})
    base["weekday"] = base["ts_hour"].dt.weekday
    base["hour"]    = base["ts_hour"].dt.hour
    Xy = base.merge(slot_top, on="ts_hour", how="left").dropna(subset=["label"]).copy()

    # 지난주 같은 시각 라벨(없으면 OTHER)
    lag7 = Xy[["ts_hour","label"]].copy()
    lag7["ts_hour"] = lag7["ts_hour"] + pd.Timedelta(days=7)
    Xy = Xy.merge(lag7.rename(columns={"label":"label_lag7"}), on="ts_hour", how="left")
    Xy["label_lag7"] = Xy["label_lag7"].fillna("OTHER")

    # 라벨 축소: 상위 K개만 유지, 나머지 OTHER
    top = Xy["label"].value_counts().nlargest(topK).index
    Xy["y"] = np.where(Xy["label"].isin(top), Xy["label"], "OTHER")

    # 범주형 원-핫(weekday, hour, label_lag7)
    cat = Xy[["weekday","hour","label_lag7"]].astype(str)
    X = pd.get_dummies(cat, drop_first=False)

    y = Xy["y"].values
    meta = Xy[["ts_hour","weekday","hour","label","label_lag7"]].copy()
    return meta, X, y
